In [374]:

import numpy as np

import pandas as pd


LIDAR_MAX_RANGE=20

In [375]:
'''
Read data from txt file using pandas
'''

df = pd.read_csv('scans.txt', delimiter = "\n")

#print(df)

In [376]:
'''
Creating a Theta vector
'''
theta=np.arange(180)
#print(theta)

In [377]:
'''
Storing the first column of the data fram to be the Lidar readings vector
'''
scanned_data = df.iloc[:, 0].values
print(scanned_data.shape)
#scanned_data=scanned_data.reshape(180,1)

(180,)


In [378]:
'''
Element wise multiplication of Theta and Lidar readings vectors in order to get the polar coordinates
Used numpy instead of making loops as numpy is fairly faster on bigger vectors, won't make big differnece on a 180 elemnt vector though
'''
cartesian= np.zeros(shape=(2, 180))

cartesian[0]= scanned_data*np.cos((theta*np.pi)/180)
cartesian[0]+=LIDAR_MAX_RANGE
cartesian[1]= scanned_data*np.sin((theta*np.pi)/180)
cartesian=np.floor(cartesian.T)
# j=0
# for i in range(0,180):
#     if(scanned_data[i]>0):
#         cartesian[j]=scanned_data[i]*np.cos((i*np.pi)/180),scanned_data[i]*np.sin((i*np.pi)/180)
#         j+=1

#cartesian=np.floor(cartesian)
#print(cartesian)


In [379]:
'''
Creating the 2D grid and a 2d array 
'''

#grid is columns*rows 
#meaning that to index the grid it will be grid[y_coordinate][x_coordinate]
# grid shape is (height,width)=(lidar_max_range,2*lidar_max_range)
grid= np.zeros(shape=(LIDAR_MAX_RANGE,2*LIDAR_MAX_RANGE))


In [380]:
'''
This cell is to fill the grid with 1s in the place of objects seen by LIDAR
'''

for i in cartesian:
    x,y=i
    grid[int(y),int(x)]=1

binary_image=np.copy(grid)

In [381]:
def grid_index_exist(x,y):
    '''
    Function takes the x,y positoin of the grid and retruns whether postions is in bounds or not

    Parameters
    ----------
    x : integer
        x postion of the cell 
    
    y: integer
        y positoin of the cell

    '''
    max_x,max_y=grid.shape
    if(x>=0 and x<max_x and y >=0 and y<max_y):
        return 1
    return 0

In [382]:
delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

# for i in delta:
#     print(i[0]," ",i[1])

In [383]:
#visited array to be used as flags
visited= np.zeros(shape=(LIDAR_MAX_RANGE,2*LIDAR_MAX_RANGE))
# delta array is used to get the 8 neigghbours to a cell
delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

#ressetting the grid to the binary_image in case the grid was labelled before
grid=np.copy(binary_image)

In [384]:

#x,y , x for row and y for column 
#unlike the real map having x for width and y for height 



def DFS (x,y,CC):
    '''
    takes the x,y position of the cell and check whether it has neighbours in order to mark them, using DFS

    Parameters
    ----------
    x : integer
        x postion of the cell 
    
    y: integer
        y positoin of the cell

    CC : integer
        the value to use inorder to mark the cells and its neighbours if exist any



    grid axees are as following   (rows,columns)=(x,y)
    -----------------------------------------y axis--------------------------------
    |
    |
    |
    |
    |
    x

    a
    x
    i
    s
    |
    |
    |
    |
    |
    '''
    visited[x][y]=1
    grid[x][y]=CC
    print("IN DFS")
    for neighbour in delta:
        new_x,new_y=x+neighbour[0],y+neighbour[1]
        if(grid_index_exist(new_x,new_y)):
            if(visited[new_x][new_y]==0 and grid[new_x][new_y]>0):
                print(new_x,' ',new_y)
                DFS(new_x,new_y,CC)
    
    return  


In [385]:
'''
Running DFS on the grid to number the connected components together
'''

delta=np.array([[-1,-1],[-1, 0],[-1, 1],[ 0,-1],[ 0, 1], [ 1,-1],[ 1, 0],[ 1, 1]])

CC=int(0)
print(grid.shape)
outter_loop,inner_loop=grid.shape

for i in range(0,outter_loop):
    for j in range (inner_loop):
        if(visited[i,j]==0 and grid[i,j]==1):
            CC+=1
            #CALL DFS and label the cells with CC meanwhile searching
            DFS(i,j,CC)
            pass
        pass


(20, 40)
IN DFS
0   10
IN DFS
1   9
IN DFS
IN DFS
IN DFS
0   30
IN DFS
0   31
IN DFS
1   30
IN DFS
1   29
IN DFS
IN DFS
6   12
IN DFS
7   11
IN DFS
7   12
IN DFS
8   12
IN DFS
IN DFS
6   27
IN DFS
7   27
IN DFS
8   28
IN DFS
IN DFS
15   18
IN DFS
15   19
IN DFS
15   20
IN DFS
16   18
IN DFS


In [386]:
# outter_loop,inner_loop=grid.shape
# print(outter_loop,"   ",inner_loop)
# print(visited.shape)
# print(visited)
# print("###################################################3")
# print(grid)

In [387]:
'''
This cell is to print the output
'''

output_file=open("out.txt","w")

for vector in grid:
    for element in vector:
        #temp=element.tostring()
        #print(str(element))
        if(element==0):
            output_file.write(".")
        else:
            output_file.write(str(int(element)))
        output_file.write(" ")
    output_file.write("\n")

output_file.close()
    